In [62]:
# from unittest import skip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn.model_selection import train_test_split
import sklearn 
import scipy.stats as st
    
# help(sklearn)

a = [3,3.5,2, 1.8, 1]
b = st.norm(3,0.8).rvs(100)
# print(help(st.norm))
print(np.mean(a), np.std(a))
r = st.shapiro(a)
r2 = st.shapiro(b)
print(r,r2)
ret =  st.ttest_1samp(a,2.0, alternative='less')
r_eqv = st.levene(b, a)
r_eqv2 = st.bartlett(b, a)
print(r_eqv,r_eqv2)
ret =  st.ttest_ind(a,b, equal_var=True )
print(ret)
 

2.2600000000000002 0.8890444308357147
ShapiroResult(statistic=0.963385283946991, pvalue=0.8313377499580383) ShapiroResult(statistic=0.9833040237426758, pvalue=0.23821096122264862)
LeveneResult(statistic=0.08895495868488283, pvalue=0.7661107657004158) BartlettResult(statistic=0.23501151808713103, pvalue=0.6278325065581375)
TtestResult(statistic=-2.18277445032824, pvalue=0.03132390023802373, df=103.0)


In [26]:
import scipy
from statsmodels.tsa import arima,api 

ll = [i for i in dir(scipy.stats) if "ba" in i]
ll


['_mstats_basic',
 'barnard_exact',
 'bartlett',
 'bayes_mvs',
 'crystalball',
 'mstats_basic']

## **4 전처리** 

In [14]:
# 기본적인 모듈, EDA시 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
import statsmodels.tsa

# 전처리 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
# pca = PCA( n_components = 4) 
# pca_fit = pca.fit(x)
# print("고유 값 : ", pca.singular_values_)
# print("분산 설명력: ", pca.explained_variance_ratio_)

ll = [i for i in dir(sklearn.metrics) if "error" in i ]
stats = [i for i in dir(statsmodels.tsa)]
# 


In [122]:
## 불군형 처리 
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler

x, y = make_classification(n_samples=2000, n_features=5, weights=[0.3], flip_y=0, n_classes=2)
# x, y = make_classification(n_samples=2000,n_features=5,n_informative=3, n_clusters_per_class=1, n_classes=3,weights=[0.5,0.2])

# x = pd.DataFrame(x)
# y = pd.DataFrame(y)
s = Counter(y)
print(type(s),s) 

# print(x.head())

# import matplotlib.pyplot as plt

# fig,ax = plt.subplots(5,1)

# ax[0].scatter(y, x[0])
# ax[1].scatter(y, x[1])
# ax[2].scatter(y, x[2])
# ax[3].scatter(y, x[3])
# ax[4].scatter(y, x[4])
# plt.show()

# help(make_classification)

<class 'collections.Counter'> Counter({1: 1400, 0: 600})


In [148]:
#언더샘플링
# undersample = RandomUnderSampler(sampling_strategy='majority')
undersample = RandomUnderSampler(sampling_strategy='auto')
x_under, y_under = undersample.fit_resample(x, y)
print(Counter(y_under))
# undersample = RandomUnderSampler(sampling_strategy=0.5) 
# x_under2, y_under2 = undersample.fit_resample(x, y)
# print(Counter(y_under2))

Counter({0: 600, 1: 600})


In [153]:
# 오버 샘플링
from imblearn.over_sampling import RandomOverSampler
# oversample = RandomOverSampler(sampling_strategy=0.5) 
oversample = RandomOverSampler(sampling_strategy='auto')
x_over, y_over = oversample.fit_resample(x, y)

print(Counter(y_over))

# help(RandomOverSampler)

Counter({1: 1400, 0: 1400})


In [154]:
#SMOTE
from imblearn.over_sampling import SMOTE
smote_sample = SMOTE(sampling_strategy='minority') 
x_sm, y_sm = smote_sample.fit_resample(x, y)
print(Counter(y_sm))

Counter({1: 1400, 0: 1400})


### **6 머신러닝**

In [7]:
# 머신러닝 - 회귀분석 
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.linear_model import SGDRegressor
# 다항회귀
from sklearn.preprocessing import PolynomialFeatures
# poly_reg=PolynomialFeatures(degree=2)
# X_poly=poly_reg.fit_transform(X_train.reshape(-1,1))

# 리지, 라소, 엘라스틱 
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# 소프트맥스 회귀 
from sklearn.linear_model import LogisticRegression
# softm=LogisticRegression(multi_class='multinomial', solver='lbfgs', C=10)
# softm.fit(train_x, train_y)

# 머시너링 - 앙상블 
 ## 랜덤포레스트 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
# clf = BaggingClassifier(base_estimator =DecisionTreeClassifier())
# pred = clf.fit(x_train, y_train).predict(x_test)
# print("Accuracy Score : ", clf.score(x_test, pred))

## 배깅
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
# reg = BaggingRegressor(base_estimator =DecisionTreeRegressor(), oob_score=True)
# reg=reg.fit(X, y)
# reg.oob_score_

## 부스팅 
from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier(base_estimator =None)
# pred=clf.fit(x_train, y_train).predict(x_test)
# print("정확도 : ", clf.score(x_test, y_test))
from sklearn.ensemble import AdaBoostRegressor
# reg = AdaBoostRegressor(base_estimator =None)
# pred=reg.fit(x_train, y_train).predict(x_test)

# 머신러닝 - SVM 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR
# clf = SVC(C=0.5)
# clf.fit(train_x, train_y)

# KNN - 분류기
from sklearn.neighbors import KNeighborsClassifier
# KNN - 회귀식
from sklearn.neighbors import KNeighborsRegressor

# Decision Tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
# clf=DecisionTreeClassifier(max_depth =5)
# clf.fit(x_train, y_train)
from sklearn.tree import export_graphviz
# dt_dot_data = export_graphviz(clf, feature_names = feature_names,
#                               class_names = target_names,
#                               filled=True, rounded =True,
#                               special_characters=True)
# dt_graph=pydotplus.graph_from_dot_data(dt_dot_data)

# GaussianNB
from sklearn.naive_bayes import GaussianNB
# gnb = GaussianNB()
# pred = gnb.fit(x_train, y_train).predict(x_test)
# print("Accuracy Score : ", gnb.score(x_test, y_test))

#베르누이 
from sklearn.feature_extraction.text import CountVectorizer 
# cv = CountVectorizer(binary =True)
# x_traincv = cv.fit_transform(x_train)
# x_traincv.shape
from sklearn.naive_bayes import BernoulliNB
# bnb = BernoulliNB()
# bnb.fit(x_traincv, y_train)
# 머신러닝 - 평가 지표 

# 다항 나이브베이즈 
from sklearn.naive_bayes import MultinomialNB
# mnb = MultinomialNB()
# mnb.fit(x_traincv, y_train)

# 평가지표 
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score


In [3]:
import sklearn.linear_model as lm

# print(dir(lm))

# print(help(lm.QuantileRegressor))
lms = {"ARDRegression":"Bayesian ARD regression","BayesianRidge":"Bayesian ridge regression",
       "ElasticNet":"Linear regression with combined L1 and L2 priors as regularizer",
  "ElasticNetCV":"Elastic Net model with iterative fitting along a regularization path",
  "GammaRegressor":"Generalized Linear Model with a Gamma distribution",
  "HuberRegressor":"L2-regularized linear regression model that is robust to outliers",
  "Lasso":"Linear Model trained with L1 prior as regularizer", 
  "PassiveAggressiveClassifier":"Passive Aggressive Classifier",
       "PoissonRegressor":"Generalized Linear Model with a Poisson distribution",
  "SGDRegressor":"Linear model fitted by minimizing a regularized empirical loss with SGD",
"QuantileRegressor":"Linear regression model that predicts conditional quantiles",
"RANSACRegressor":"이상치에 강건한 회귀모델", 
"TheilSenRegressor":"이상치에 강건한 회귀모델, Theil-Sen 추정량(Theil-Sen Estimator)은 중앙값을 사용하여 추정되는 선형 회귀 모델입니다",
"TweedieRegressor":"Tweedie 분포를 기반으로 한 회귀 모델", "ridge_regression":"ridge regression"
}
print(lms)


{'ARDRegression': 'Bayesian ARD regression', 'BayesianRidge': 'Bayesian ridge regression', 'ElasticNet': 'Linear regression with combined L1 and L2 priors as regularizer', 'ElasticNetCV': 'Elastic Net model with iterative fitting along a regularization path', 'GammaRegressor': 'Generalized Linear Model with a Gamma distribution', 'HuberRegressor': 'L2-regularized linear regression model that is robust to outliers', 'Lasso': 'Linear Model trained with L1 prior as regularizer', 'PassiveAggressiveClassifier': 'Passive Aggressive Classifier', 'PoissonRegressor': 'Generalized Linear Model with a Poisson distribution', 'SGDRegressor': 'Linear model fitted by minimizing a regularized empirical loss with SGD', 'QuantileRegressor': 'Linear regression model that predicts conditional quantiles', 'RANSACRegressor': '이상치에 강건한 회귀모델', 'TheilSenRegressor': '이상치에 강건한 회귀모델, Theil-Sen 추정량(Theil-Sen Estimator)은 중앙값을 사용하여 추정되는 선형 회귀 모델입니다', 'TweedieRegressor': 'Tweedie 분포를 기반으로 한 회귀 모델', 'ridge_regression'

## **7.1 통계처리**

In [39]:
# 통계 처리 

import scipy
import scipy.stats as st 

from scipy.stats import bernoulli, norm, binom, nbinom, geom, hypergeom,multinomial

norm(10,2).pdf(11)

0.17603266338214976

## **7.2 t-test**

In [ ]:
# t-test 
# 정규성, 등분산 검정 
from scipy.stats import shapiro, levene, bartlett 
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind

## **7.3 anova - 1원배치 분산분석**

In [ ]:
# anova 1원 배치 

# stats.kruskal(setosa,versicolor,virginica)
#정규성 검사 
from scipy.stats import shapiro

# 정규성 만족하지 않으면 
from scipy.stats import kruskal 
#stats.kruskal(setosa,versicolor,virginica)

#정규성 만족하면, 등분산 검사 
from scipy.stats import bartlett,levene

# stats.levene(setosa,versicolor,virginica)
# 등분산을 만족하면 
from scipy.stats import f_oneway
# stats.f_oneway(setosa,versicolor,virginica)
# 등분산을 만족하지 않으면 
import pingouin as pg 
#pg.welch_anova(data = Iris_data, dv ='sepal width', between='target')
# 사후검정 
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
# mc = MultiComparison(data= Iris_data["sepal width"], groups=Iris_data["target"] )
# tuekeyhsd = mc.tukeyhsd(alpha=0.05)
# fig = tuekeyhsd.plot_simultaneous()
#tuekeyhsd.summary()


## **7.3 anova - 2원배치 분산 분석**

In [6]:
# anova 2원 배치 
## 분산분석 수행 
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
# formula = 'mpg ~ C(cyl) + C(am) + C(cyl):C(am)'
# model = ols(formula, mtcars).fit()
# aov_table = anova_lm(model, typ=3)
print(dir(anova_lm))

from statsmodels.graphics.factorplots import interaction_plot
#import matplotlib.pyplot as plt
## 독립변수 cyl,am와 종속변수 mpg을 Series로 변경 
# cyl = mtcars["cyl"]
# am = mtcars["am"]
# mpg = mtcars["mpg"]
# fig, ax = plt.subplots(figsize=(6, 6))
# fig = interaction_plot(cyl,am, mpg,
#                        colors=['red', 'blue'], markers=['D', '^'], ms=10, ax=ax)

['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


## **7.4 교차분석**


In [5]:
#교차분석 
#적합도 거정 
# 유의수준 0.05로 적합도 검정 수행
from scipy.stats import chisquare
# chi = chisquare(table, f_exp=[171,171])
# print('<적합도 검정>\n',chi)

# 카이제곱 검정을 통한 독립성 검정 수행
from scipy.stats import chi2_contingency
import pandas as pd
# 데이터 불러오기
df = pd.read_csv("../data/titanic.csv")
table = pd.crosstab(df['class'], df['survived'])
chi, p, df, expect = chi2_contingency(table) 
print('Statistic:', chi)
print('p-value:', p)
print('df:', df)
print('expect: \n', expect)

FileNotFoundError: [Errno 2] No such file or directory: '../data/titanic.csv'

## **7.5 회귀분석**

In [ ]:
from statsmodels.formula.api import ols
# y = house['price']
# X = house[['sqft_living']]
# # 단순선형회귀모형 적합
# lr = ols('price ~ sqft_living',data=house).fit()
# y_pred = lr.predict(X)
# lr.summary()

from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 독립변수와 종속변수를 데이터프레임으로 나누어 저장하는 함수 
# y,X = dmatrices("Price ~ EngineSize + RPM + Weight+ Length + MPGcity + MPGhighway",
#                 data = Cars,return_type ="dataframe")
# # 독립변수끼리의 VIF값을 계산하여 데이터프레임으로 만드는 과정 
# vif_list = []
# for i in range(1,len(X.columns)): 
#     vif_list.append([variance_inflation_factor(X.values,i), X.columns[i]])
# pd.DataFrame(vif_list,columns=['vif','variable'])
# model = smf.ols(formula ="Price ~ EngineSize + RPM + Weight  + MPGhighway", data = Cars)
# result = model.fit()
# result.summary()



In [ ]:
# 전진 선택법 
import time
import itertools
def processSubset(X,y, feature_set):
            model = sm.OLS(y,X[list(feature_set)]) # Modeling
            regr = model.fit() # 모델 학습
            AIC = regr.aic # 모델의 AIC
            return {"model":regr, "AIC":AIC}
# 전진선택법
def forward(X, y, predictors):
    # 데이터 변수들이 미리정의된 predictors에 있는지 없는지 확인 및 분류
    remaining_predictors = [p for p in X.columns.difference(['Intercept']) if p not in predictors]
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X=X, y= y,            feature_set=predictors+[p]+['Intercept']))
        
    # 데이터프레임으로 변환
    models = pd.DataFrame(results)
    # AIC가 가장 낮은 것을 선택
    best_model = models.loc[models['AIC'].argmin()] # index
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in")
    print('Selected predictors:',best_model['model'].model.exog_names,' AIC:',best_model[0] )
    return best_model


In [ ]:
# 후진소거법
def backward(X,y,predictors):
    tic = time.time()
    results = []
    
    # 데이터 변수들이 미리정의된 predictors 조합 확인
    for combo in itertools.combinations(predictors, len(predictors) -1):
        results.append(processSubset(X=X, y= y,        feature_set = list(combo)+['Intercept']))
    models = pd.DataFrame(results)
    
    # 가장 낮은 AIC를 가진 모델을 선택
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on",          len(predictors) -1, "predictors in", (toc - tic))
    print('Selected predictors:',best_model['model'].model.exog_names,         'AIC:',best_model[0] )

    return best_model

In [ ]:
# 단계적 선택법
def Stepwise_model(X,y):
    Stepmodels = pd.DataFrame(columns=["AIC", "model"])
    tic = time.time()
    predictors = []
    Smodel_before = processSubset(X,y,predictors+['Intercept'])['AIC']

    for i in range(1, len(X.columns.difference(['Intercept'])) +1):
        Forward_result = forward(X=X, y=y, predictors=predictors) 
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]["model"].model.exog_names
        predictors = [ k for k in predictors if k !='Intercept']
        Backward_result = backward(X=X, y=y, predictors=predictors)

        if Backward_result['AIC']< Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]["model"].model.exog_names
            Smodel_before = Stepmodels.loc[i]["AIC"]
            predictors = [ k for k in predictors if k !='Intercept']
            print('backward')

        if Stepmodels.loc[i]['AIC']> Smodel_before:
            break
        else:
            Smodel_before = Stepmodels.loc[i]["AIC"]
    toc = time.time()
    print("Total elapsed time:", (toc - tic), "seconds.")

    return (Stepmodels['model'][len(Stepmodels['model'])])


In [ ]:
Stepwise_best_model = Stepwise_model(X=X, y=y)
Stepwise_best_model.summary()

## **7.6 군집분석**

In [ ]:
## 군집분석 
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
# calinsk_harabasz_score
# for k in range(2, 10):
#     kmeans_model = KMeans(n_clusters=k, random_state=1).fit(X)
#     labels = kmeans_model.labels_
#     print(calinski_harabasz_score(X, labels))
# elbow
# for i in range(1, 11):
#     km=KMeans(n_clusters=i, random_state=1)
#     km.fit(X)
#     sse.append(km.inertia_) # km.inertia_

from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture 
# iris = pd.read_csv("../data/iris.csv")
# iris.info()
# df = iris.drop("target", axis=1)
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)

# gmm = GaussianMixture(n_components=3)
# gmm.fit(df_scaled)
# gmm_labels = gmm.predict(df_scaled)
# gmm_labels

# statsmodels.formula.api vs statsmodels.api 의 차이는 
 전자는 자동으로 상수항을 추가해 준다면, 후자는 x_new = sm.add_constant(x) 과 같이 처리해주고 SM_model_2 = sm.OLS(y, x_new).fit() 이렇게 해주어야 한다. 
 https://michael-fuchs-python.netlify.app/2019/07/02/statsmodel-formula-api-vs-statsmodel-api/

연관 분석(association analysis)에서 "lift(향상도)"는 상품 간의 연관성을 측정하는 데 사용되는 지표 중 하나입니다. 
<span style="color:red"> 향상도는 두 개의 상품 간의 관계가 랜덤하게 발생한 경우에 비해 그 빈도가 얼마나 높은지를 측정합니다.</span>
일반적으로 향상도는 다음과 같은 공식으로 계산됩니다:
향상도의 해석은 다음과 같습니다:
향상도가 1보다 크면, 두 상품 간의 연관 관계가 랜덤한 경우보다 더 자주 함께 발생한다는 것을 의미합니다. 이는 양의 상관 관계를 나타냅니다.
향상도가 1에 가까우면, 두 상품 간의 연관성이 랜덤한 경우와 비슷하다는 것을 의미합니다.
향상도가 1보다 작으면, 두 상품 간의 연관 관계가 랜덤한 경우보다 덜 자주 함께 발생한다는 것을 의미합니다. 이는 음의 상관 관계를 나타냅니다.
따라서 향상도는 연관 규칙의 유용성을 평가하는 데 사용되며, 특정 상품이 다른 상품과 어떻게 관련되어 있는지를 파악하는 데 도움이 됩니다.

## **7.7 연관분석**

In [52]:
## 연관분석 
#runstest_1samp, apriori, assocaiton_rules  
# 연관성에 대한 검정 
from statsmodels.sandbox.stats.runs import runstest_1samp
# 연관규칙분석 
#  Lift 해석은 1보다 크며, 연관이 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [71]:
import matplotlib.pyplot as plt
df =pd.DataFrame(np.random.randint(100,1000,100))
ss = [["a","b","c"],["d","e"],["a","b","c","d","e"]]

from  mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit_transform(ss)
# print(help(te))
print(type(te), te.columns_, te_ary)
print(te.inverse_transform(te_ary))

df = pd.DataFrame(te_ary, columns = te.columns_)
print(df)

# df.plot()
# plt.show()

<class 'mlxtend.preprocessing.transactionencoder.TransactionEncoder'> ['a', 'b', 'c', 'd', 'e'] [[ True  True  True False False]
 [False False False  True  True]
 [ True  True  True  True  True]]
[['a', 'b', 'c'], ['d', 'e'], ['a', 'b', 'c', 'd', 'e']]
       a      b      c      d      e
0   True   True   True  False  False
1  False  False  False   True   True
2   True   True   True   True   True


## **7.8 시계열 분석**

In [ ]:
# 7-8.시계열, 이원분석 anova, lm_anova, tsa,
from statsmodels.tsa.seasonal import seasonal_decompose
# ts = data
# result = seasonal_decompose(ts, model='multiplicative')
# plt.rcParams['figure.figsize'] = [12, 8]
# result.plot()
# plt.show()

## 정상성 
from statsmodels.tsa.stattools import adfuller
# training = data[:"2016-12-01"]
# test = data.drop(training.index)
# print(help(adfuller))

# 정상성 검사 
adf = adfuller(training, regression='ct')
print(adf)
from statsmodels.api import PCA, OLS, MANOVA
from statsmodels.formula.api import ols 
import statsmodels.tsa 
from statsmodels.tsa import arima,api 

# PACF, ACF

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

from statsmodels.tsa.arima.model import ARIMA
# model = ARIMA(training, order=(2,1,2))
# res = model.fit()
# res.summary()
# plt.plot(res.predict())
# plt.plot(training)
# plt.legend(["P","R"])
# plt.show()


In [ ]:
# 7-8.
from pmdarima import auto_arima
auto_model = auto_arima(training, start_p=0, d=1, start_q=0,
                        max_p=3, max_q=3, 
                        start_P=0, start_Q=0,
                        max_P=3, max_Q=3, m=12,
                        seasonal=True, information_criterion='aic',
                        trace=True)
# m =12면 달 기준, m  =1 이면 년 단위 
auto_model.summary()
auto_pred_y= pd.DataFrame(auto_model.predict(n_periods=len(test)), 
                          index=test.index) 
auto_pred_y.columns = ['predicted_price']
auto_pred_y

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print("r2_score : ", r2_score(test_y, auto_pred_y))
RMSE = mean_squared_error(test_y, auto_pred_y)**0.5
print("RMSE : " , RMSE)